In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import zipfile

In [2]:
pip install kaggle

In [3]:
!pip install --upgrade tensorflow-gpu

Requirement already up-to-date: tensorflow-gpu in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [4]:
!ls -a

.  ..  .config	.kaggle  sample_data


In [5]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"envykhamidov","key":"fdd2921f97a333303ae41722b3421d9f"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)


In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [9]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [10]:
!kaggle competitions download -c chinese-char-recognition-smmo19

 88% 145M/165M [00:01<00:00, 106MB/s] 
100% 165M/165M [00:01<00:00, 132MB/s]
 91% 169M/185M [00:02<00:00, 60.4MB/s]
100% 185M/185M [00:02<00:00, 80.3MB/s]
 87% 169M/195M [00:02<00:00, 71.3MB/s]
100% 195M/195M [00:02<00:00, 94.2MB/s]
 96% 193M/202M [00:01<00:00, 118MB/s]
100% 202M/202M [00:01<00:00, 120MB/s]
 80% 90.0M/113M [00:00<00:00, 53.5MB/s]
100% 113M/113M [00:00<00:00, 122MB/s]  
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 136MB/s]


In [0]:
a1 = zipfile.ZipFile('/content/{/content}/competitions/chinese-char-recognition-smmo19/test.npy.zip', 'r')
a2 = zipfile.ZipFile('/content/{/content}/competitions/chinese-char-recognition-smmo19/train-1.npy.zip', 'r')
a3 = zipfile.ZipFile('/content/{/content}/competitions/chinese-char-recognition-smmo19/train-2.npy.zip', 'r')
a4 = zipfile.ZipFile('/content/{/content}/competitions/chinese-char-recognition-smmo19/train-3.npy.zip', 'r')
a5 = zipfile.ZipFile('/content/{/content}/competitions/chinese-char-recognition-smmo19/train-4.npy.zip', 'r')


In [0]:
a1.extractall()
a2.extractall()
a3.extractall()
a4.extractall()
a5.extractall()

In [0]:
data_train = np.load("./train-1.npy", allow_pickle=True)
for i in range(2, 5):
    t = np.load(f"./train-{i}.npy", allow_pickle=True)
    data_train = np.concatenate([data_train, t])


In [0]:
data_test = np.load("./test.npy", allow_pickle=True)


In [0]:
a = np.arange(0,1000)
b = np.unique(data_train[:, 1])
#for i in range(len(data_train[:, 1])):
 # for j in range(len(b)):
  #  if data_train[i, 1] == b[j]:
     # data_train[i , 1] = a[j]

In [0]:
slovar = {}
temp = np.eye(1000)
for i in range(len(b)):
  slovar[b[i]] = temp[i]

In [17]:
from sklearn.model_selection import train_test_split
X_train_, X_test_, y_train_, y_test_ = train_test_split(data_train[:, 0], data_train[:, 1], test_size=0.1, random_state=42)
X_train = X_train_.reshape(-1, 1) 
X_test  = X_test_.reshape(-1, 1)  
y_train = y_train_.reshape(-1, 1)  
y_test  = y_test_.reshape(-1, 1) 
raw_train = np.concatenate([X_train, y_train], axis = 1)
raw_test = np.concatenate([X_test, y_test], axis = 1)
raw_train.shape, raw_test.shape

((299688, 2), (33299, 2))

In [0]:

tf.compat.v1.disable_tensor_equality()

In [0]:
IMG_SIZE = 64
def preprocess_train(image, label):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  
    return image, label

In [0]:
batch_size = 32
def train_gen():
    for img, label in raw_train:
        img = img[..., None] # [batch, w, h, channels]
        yield img, slovar[label]

ds_train = tf.data.Dataset.from_generator(train_gen,
                                          output_types=(tf.float32, tf.int32),
                                          output_shapes=((None,None,1), (1000))
                                         ).map(preprocess_train, num_parallel_calls=-1).prefetch(-1).shuffle(1024).batch(batch_size).repeat()

In [0]:
batch_size = 32
def train_gen1():
    for img, label in data_train:
        img = img[..., None] # [batch, w, h, channels]
        yield img, slovar[label]

train_dataset = tf.data.Dataset.from_generator(train_gen1,
                                          output_types=(tf.float32, tf.int32),
                                          output_shapes=((None,None,1), (1000))
                                         ).map(preprocess_train, num_parallel_calls=-1).prefetch(-1).shuffle(1024).batch(batch_size).repeat()

In [0]:
def train_gen2():
    for img, label in raw_test:
        img = img[..., None] # [batch, w, h, channels]
        yield img, slovar[label]

ds_test = tf.data.Dataset.from_generator(train_gen2,
                                          output_types=(tf.float32, tf.int32),
                                          output_shapes=((None,None,1), (1000))
                                         ).map(preprocess_train, num_parallel_calls=-1).prefetch(-1).shuffle(1024).batch(batch_size).repeat()


In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.models import load_model

Using TensorFlow backend.


In [0]:
import keras
from keras import initializers

In [0]:
model = tf.keras.Sequential([

tf.keras.layers.Conv2D(filters=16, padding='same', kernel_size=(3,3), input_shape=(64,64, 1),
activation='relu', kernel_initializer = 'glorot_normal'),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=(3,3), 
activation='relu', kernel_initializer = 'glorot_normal'),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3),
activation='relu', kernel_initializer = 'glorot_normal'),

tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),

tf.keras.layers.Dropout(0.2),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),

tf.keras.layers.Dropout(0.2),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=256, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),

tf.keras.layers.Dropout(0.2),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=256, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.BatchNormalization(),
tf.keras.layers.Conv2D(filters=512, padding='same', kernel_size=(3,3), kernel_initializer='glorot_normal',
activation='relu'),

tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'),

tf.keras.layers.Dropout(0.2),

tf.keras.layers.Flatten(),

tf.keras.layers.Dense(4096, activation='relu'),
tf.keras.layers.Dropout(0.25),

tf.keras.layers.Dense(2048, activation='relu'),
tf.keras.layers.Dropout(0.25),

tf.keras.layers.Dense(1000, activation = 'softmax')])




In [0]:
import math

In [0]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.losses = []
       self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
       self.losses.append(logs.get('loss'))
       self.lr.append(step_decay(len(self.losses)))
       print("\n", ' lr:', step_decay(len(self.losses)))


def step_decay(epoch):
   initial_lrate = 0.0005
   lrate = initial_lrate * 0.9 ** epoch
   return lrate
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

loss_history = LossHistory()
callbacks_list = [loss_history, lrate]
   

In [0]:

model.compile(loss="categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0), metrics=["accuracy"])

In [31]:
model.fit(ds_train, steps_per_epoch= 1000,
           epochs=60, shuffle=True, validation_data = ds_test, validation_steps = 1000, use_multiprocessing = True,
         verbose = 1, callbacks = [loss_history, lrate])

Train for 1000 steps, validate for 1000 steps
Epoch 1/60
 999/1000 [============================>.] - ETA: 0s - loss: 6.7299 - accuracy: 0.0039
  lr: 0.00045000000000000004
1000/1000 [==============================] - 43s 43ms/step - loss: 6.7297 - accuracy: 0.0040 - val_loss: 6.1214 - val_accuracy: 0.0090
Epoch 2/60
 999/1000 [============================>.] - ETA: 0s - loss: 5.3575 - accuracy: 0.0422
  lr: 0.00040500000000000003
1000/1000 [==============================] - 34s 34ms/step - loss: 5.3568 - accuracy: 0.0422 - val_loss: 3.9270 - val_accuracy: 0.1574
Epoch 3/60
 998/1000 [============================>.] - ETA: 0s - loss: 3.1299 - accuracy: 0.2832
  lr: 0.0003645000000000001
1000/1000 [==============================] - 34s 34ms/step - loss: 3.1276 - accuracy: 0.2836 - val_loss: 1.8129 - val_accuracy: 0.5415
Epoch 4/60
 999/1000 [============================>.] - ETA: 0s - loss: 1.4972 - accuracy: 0.6109
  lr: 0.00032805000000000003
1000/1000 [==============================]

In [0]:
def preprocess_train1(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  
    return image

def train_gen3():
    for img in data_test:
        img = img[..., None] # [batch, w, h, channels]
        yield img

test_dataset = tf.data.Dataset.from_generator(train_gen3,
                                          output_types=(tf.float32),
                                          output_shapes=((None,None,1)
                                         )).map(preprocess_train1, num_parallel_calls=-1).prefetch(-1).batch(batch_size)



In [33]:
preds = model.predict(test_dataset, verbose = 1)
preds

2602/2602 [==============================] - 29s 11ms/step


array([[2.9500943e-29, 5.4592164e-27, 2.2573682e-26, ..., 2.4409804e-28,
        1.7933741e-30, 1.0734788e-21],
       [2.9998174e-22, 4.1382201e-26, 2.9018090e-27, ..., 9.3723807e-18,
        1.9840636e-18, 4.4558329e-18],
       [5.5996799e-23, 2.8356441e-33, 7.5742780e-21, ..., 3.5577068e-36,
        1.1240191e-34, 1.3059117e-33],
       ...,
       [5.1629688e-19, 9.4464582e-11, 1.6449226e-20, ..., 1.7336350e-15,
        1.5423432e-13, 2.2155753e-17],
       [3.9959623e-30, 9.5887381e-23, 7.4763762e-32, ..., 2.0597079e-16,
        1.5227912e-20, 5.8867466e-18],
       [1.3883081e-23, 5.2377914e-19, 3.9297762e-23, ..., 9.3265127e-12,
        5.7775756e-10, 1.0420235e-13]], dtype=float32)

In [0]:
c = np.arange(0,1000)
slovar1 = {}
for i in range(1000):
  slovar1[c[i]] = b[i]

In [0]:
j = []
predictions = np.argmax(preds, axis = 1)
for i in predictions:
  j.append(slovar1[i])


In [40]:

itog_pred = np.concatenate([np.arange(1, 83248).reshape(-1,1), np.array(j).reshape(-1,1)], axis = 1)
itog_pred.shape

(83247, 2)

In [0]:
np.savetxt('kaggle18.csv', itog_pred, delimiter = ',')